# Реализация алгоритма Advantage-Actor Critic (A2C) (вплоть до 10 баллов)

#### дедлайн задания (сразу жёсткий): 26 апреля, 23:59 UTC+3

# Работа выполнена: Ахмаджонов Мумтозбек, М05-317.

В данной работе Вам предстоит реализовать алгоритм `Advantage Actor Critic`, обучаемый на батче из сред `Atari 2600`, работающих параллельно.

Для начала будут использованы обёртки сред, реализованные в файле `atari_wrappers.py`. Эти обёртки предварительно обрабатывают наблюдения (производят преобразования размера, цвета фрейма, взятия максимума между фреймами, пропускают часть фреймов и сводят несколько фреймов в один большой) и вознаграждения. Некоторые обёртки помогают автоматически перезапустить среду и присвоить переменной `done` значение `True` в случае смерти агента. Файл `env_batch.py` включает в себя реализацию класса `ParallelEnvBatch`, позволяющего запускать несколько сред параллельно. Для создания (инициализации) среды можно воспользоваться функцией `nature_dqn_env`. Обратите внимание, что в случае использования `PyTorch` (https://pytorch.org/) без `tensorboardX` (https://github.com/lanpa/tensorboardX) потребуется самостоятельно реализовать обёртку среды, которая будет логрировать **исходные** суммарные награды, которые *исходная* среда возвращает, и переопределить реализацию функции `nature_dqn_env`. То есть настоятельно рекомендуется применить `tensorboardX`.

Псевдокод алгоритма `Advantage Actor Critic (A2C)` приведён в **Разделе 5.2.5 (Алгоритм 20)** конспекта лекций: https://arxiv.org/pdf/2201.09746.pdf

Скрипты в данной работе используют Python версию библиотеки [OpenCV](https://pypi.org/project/opencv-python/). Для запуска сред ATARI понадобится библиотека [The Arcade Learning Environment](https://github.com/Farama-Foundation/Arcade-Learning-Environment), а также библиотека [Shimmy](https://pypi.org/project/Shimmy/). Может потребоваться установка `ffmpeg` кодека. Для Unix-based операционной системы с пакетным менеджером APT может потребоваться следующая последовательность команд:
```
sudo apt-get install -y xvfb x11-utils ffmpeg python-opengl
pip install pyglet pyvirtualdisplay opencv-python tqdm numpy moviepy gymnasium[atari]
pip install gymnasium[accept-rom-license] #run 'pip install autorom; AutoROM --accept-license' if this fails
```
Для MacOS систем рекомендуется установить репозиторий [TFM](https://github.com/serrodcal-MII/TFM/tree/main) и выполнить инструкции из [README.md](https://github.com/serrodcal-MII/TFM/blob/main/README.md).

In [1]:
from tqdm.notebook import tqdm
import numpy as np
import os
from pathlib import Path
from IPython.display import HTML
from collections import defaultdict
from functools import partial

import torch
import torch.optim as opt
from torch.optim.lr_scheduler import LambdaLR
from torch.distributions import Categorical
import torch.nn.functional as F

from gymnasium.wrappers import RecordVideo

from atari_wrappers import nature_dqn_env
from runners import EnvRunner

In [2]:
# XVFB будет запущен в случае исполнения на сервере
if type(os.environ.get("DISPLAY")) is not str or len(os.environ.get("DISPLAY")) == 0:
    ! bash ../xvfb start
    os.environ['DISPLAY'] = ':1'

bash: ../xvfb: No such file or directory


In [3]:
nenvs = 8

env = nature_dqn_env("SpaceInvadersNoFrameskip-v4", nenvs=nenvs, summaries = "Tensorboard")
                                                    # nenvs -- количество параллельно запущенных сред
                                                    # данный параметр можно варьировать для баланса
                                                    # производительность итерации/надёжность Монте-Карло оценок
                                                    # помните: при уменьшении nenvs, возможно, придётся
                                                    # увеличить количество итераций оптимизации
n_actions = env.action_space.spaces[0].n
obs, _ = env.reset()
assert obs.shape == (nenvs, 4, 84, 84)
assert obs.dtype == np.float32

A.L.E: Arcade Learning Environment (version 0.8.1+53f58b7)
[Powered by Stella]
A.L.E: Arcade Learning Environment (version 0.8.1+53f58b7)
[Powered by Stella]
A.L.E: Arcade Learning Environment (version 0.8.1+53f58b7)
[Powered by Stella]


A.L.E: Arcade Learning Environment (version 0.8.1+53f58b7)
[Powered by Stella]
A.L.E: Arcade Learning Environment (version 0.8.1+53f58b7)
[Powered by Stella]
A.L.E: Arcade Learning Environment (version 0.8.1+53f58b7)
[Powered by Stella]
A.L.E: Arcade Learning Environment (version 0.8.1+53f58b7)
[Powered by Stella]
A.L.E: Arcade Learning Environment (version 0.8.1+53f58b7)
[Powered by Stella]


Следующим шагом будет реализация модели, которая выводит логиты для категориального распределения на действия и оценку на значения $V$-функции ценности. Рекомендуется использовать архитектуру модели, представленной в публикации в журнале [Nature](https://web.stanford.edu/class/psych209/Readings/MnihEtAlHassibis15NatureControlDeepRL.pdf) со следующей модификацией: вместо одного выходного слоя нужно сделать два слоя, принимающих в качестве входа выход предшествующего скрытого слоя. **Обратите внимание**, данная модель отличается от модели, предложенной в домашней работе по DQN. Рекомендуется использовать ортогональную инициализацию с параметром $\sqrt{2}$ для ядер свёрток и инициализировать смещения нулями.

In [4]:
def conv2d_size_out(size, kernel_size, stride):
    """
    общий принцип использования данной функции:
    cur_layer_img_w = conv2d_size_out(cur_layer_img_w, kernel_size, stride)
    cur_layer_img_h = conv2d_size_out(cur_layer_img_h, kernel_size, stride)
    для вычисления размерности входа полносвязного слоя
    """
    return (size - (kernel_size - 1) - 1) // stride + 1


class Flatten(torch.nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return x.reshape(x.size(0), -1)


class NatureModel(torch.nn.Module):
    def __init__(self, bsize, nchannels, height, width, n_actions, device):
        super().__init__()
        self.device = device

        self.conv1 = torch.nn.Conv2d(
            in_channels=nchannels, out_channels=32, kernel_size=8, stride=4
        )
        self.relu1 = torch.nn.ReLU()
        cur_h, cur_w = conv2d_size_out(height, 8, 4), conv2d_size_out(width, 8, 4)
        self.conv2 = torch.nn.Conv2d(
            in_channels=32, out_channels=64, kernel_size=4, stride=2
        )
        self.relu2 = torch.nn.ReLU()
        cur_h, cur_w = conv2d_size_out(cur_h, 4, 2), conv2d_size_out(cur_w, 4, 2)
        self.conv3 = torch.nn.Conv2d(
            in_channels=64, out_channels=64, kernel_size=3, stride=1
        )
        self.relu3 = torch.nn.ReLU()
        cur_h, cur_w = conv2d_size_out(cur_h, 3, 1), conv2d_size_out(cur_w, 3, 1)
        self.flatten = Flatten()
        self.fc1 = torch.nn.Linear(in_features=64 * cur_h * cur_w, out_features=512)
        self.relu4 = torch.nn.ReLU()

        self.fc_logits = torch.nn.Sequential(
            torch.nn.Linear(in_features=512, out_features=64),
            torch.nn.LeakyReLU(),
            torch.nn.Linear(in_features=64, out_features=n_actions),
        )
        self.fc_values = torch.nn.Sequential(
            torch.nn.Linear(in_features=512, out_features=64),
            torch.nn.LeakyReLU(),
            torch.nn.Linear(in_features=64, out_features=1),
        )

        self._reset_parameters()

    def _reset_parameters(self):
        torch.nn.init.orthogonal_(self.conv1.weight.data, gain=np.sqrt(2.0))
        self.conv1.bias.data.fill_(0.0)

        torch.nn.init.orthogonal_(self.conv2.weight.data, gain=np.sqrt(2.0))
        self.conv2.bias.data.fill_(0.0)

        torch.nn.init.orthogonal_(self.conv3.weight.data, gain=np.sqrt(2.0))
        self.conv3.bias.data.fill_(0.0)

        torch.nn.init.orthogonal_(self.fc1.weight.data, gain=np.sqrt(2.0))
        self.fc1.bias.data.fill_(0.0)

        torch.nn.init.orthogonal_(self.fc_logits[0].weight.data, gain=np.sqrt(2.0))
        self.fc_logits[0].bias.data.fill_(0.0)

        torch.nn.init.orthogonal_(self.fc_logits[2].weight.data, gain=np.sqrt(2.0))
        self.fc_logits[2].bias.data.fill_(0.0)

        torch.nn.init.orthogonal_(self.fc_values[0].weight.data, gain=np.sqrt(2.0))
        self.fc_values[0].bias.data.fill_(0.0)

        torch.nn.init.orthogonal_(self.fc_values[2].weight.data, gain=np.sqrt(2.0))
        self.fc_values[2].bias.data.fill_(0.0)

    def forward(self, state):
        x = torch.tensor(np.array(state), device=self.device, dtype=torch.float32)

        x = self.conv1(x)
        x = self.relu1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.conv3(x)
        x = self.relu3(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu4(x)

        logits = self.fc_logits(x)
        value = self.fc_values(x)

        x.cpu()
        del x

        return logits, value

Вам также потребуется определить и использовать политику, которая будет использовать модель выше. В то время как модель вычисляет логиты для всех действий сразу и оценку функции ценности, политика будет сэмплировать действия, а также будет вычислять их логарифм правдоподобия. Метод `Policy.act` должен возвращать словарь всех массивов, требуемых для взаимодействия со средой и обучения модели. Обратите внимание, что действия должны быть формата `Numpy.ndarray`, в то время как другие тензоры должны быть в формате, определяемом библиотекой глубокого обучения (`Torch.tensor`, например).

In [5]:
class Policy:
    def __init__(self, model):
        self.model = model

    def act(self, inputs):
        # <Реализуйте политику посредством прямого прохода модели, сэмплирования действий и вычисления их логарифмов правдоподобия>
        # Должен возвращать dict с ключами ['actions', 'logits', 'log_probs', 'values'].
        result = {}
        logits, values = self.model(inputs)
        result["logits"] = logits
        result["values"] = values.flatten().contiguous()
        dist = Categorical(logits=logits)
        result["actions"] = dist.sample()
        result["log_probs"] = dist.log_prob(result["actions"])
        result["actions"] = result["actions"].detach().cpu().numpy()
        result["entropy"] = dist.entropy()
        return result
    
    def reset(self):
        pass

Далее требуется передать среду и политику в исполнитель `EnvRunner`, который собирает частичные траектории из среды. Класс `EnvRunner` уже реализован за Вас.

Данный исполнитель взаимодействует со средой заданное количество шагов и возвращает словарь, содержащий ключи:

* 'observations' 
* 'rewards' 
* 'resets'
* 'actions'
* и другие ключи, определённые в `Policy`

по каждому из этих ключей содержится Python `list` соответствующих результатов взаимодействий со средой указанной длины $T$ &mdash; размера частичной траектории.

Для того чтобы обучить часть модели, которая предсказывает ценности состояний, требуется вычислить целевые значения ценностей. В инстанцию класса `EnvRunner` можно подать при создании по аргументу `transforms` список вызываемых объектов ("функций"), которые последовательно будут применяться к частичным траекториям после сбора самих траекторий. Следовательно, требуется реализовать и использовать вызываемый (с определённым методом `__call__`) класс `ComputeValueTargets`. Формула для вычисления целевых значений ценности простая:

$$
\hat v(s_t) = \sum_{t'=0}^{T - 1}\gamma^{t'}r_{t+t'} + \gamma^T \hat{v}(s_{t+T}),
$$

Однако, не забудьте в реализации использовать `trajectory['resets']` флаги для проверки того, следует ли добавить целевые значения ценности на следующем шаге при вычислении целевых значений ценности на текущем шаге. У вас также имеется доступ к `trajectory['state']['latest_observation']` для получения последнего наблюдения в частичной траектории &mdash; $s_{t+T+1}$.

In [6]:
class ComputeValueTargets:
    def __init__(self, policy, gamma=0.99, device=torch.device("cuda:3")):
        self.gamma = gamma
        self.device = device
        self.policy = policy

    def __call__(self, trajectory):
        # Данный метод должен модифицировать траекторию trajectory на месте через добавление
        # итеративно заполненного списка по ключу 'value_targets'.
        # <Вычислите целевые значения ценности для данных частичных траекторий>
        nsteps = len(trajectory["rewards"])
        trajectory["value_targets"] = np.zeros((nsteps, nenvs), dtype=np.float32)
        last_values = None
        with torch.no_grad():
            tmp_results = self.policy.act(trajectory["state"]["latest_observation"])
            last_values = tmp_results["values"].detach().cpu().numpy()
            del tmp_results
        mask = 1.0 - trajectory["resets"][-1]
        trajectory["value_targets"][-1] = mask * (
            self.gamma * last_values + trajectory["rewards"][-1]
        )

        for t in range(nsteps - 2, -1, -1):
            mask = 1.0 - trajectory["resets"][t]
            trajectory["value_targets"][t] = mask * (
                trajectory["rewards"][t]
                + self.gamma * trajectory["value_targets"][t + 1]
            )

        for key in trajectory:
            if key == "state":
                continue
            if key in ["logits", "values", "log_probs", "entropy"]:
                trajectory[key] = torch.stack(trajectory[key])
            elif key == "value_targets":
                # trajectory[key] = torch.tensor(np.stack(trajectory[key]), dtype=torch.float32, device=self.device)
                trajectory[key] = torch.from_numpy(trajectory[key]).to(self.device)
        return trajectory

После вычисления целевых значений ценности требуется преобразовать списки результатов взаимодействия со средой в тензоры с первой компонентой размерности `batch_size`, равной призведению `T * nenvs`, то есть требуется свести в одну компоненту первые две компоненты размерности.

In [7]:
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=2)

In [8]:
class MergeTimeBatch:
    """
    Сращивает первые две оси, обычно отвечающие за время и за инстанцию среды, соответственно.
    """
    def __call__(self, trajectory):
        # Модификация траектории на месте. 
        for key in trajectory:
            if key not in ["logits", "values", "log_probs", "entropy", "value_targets"]:
                continue
            trajectory[key] = trajectory[key].flatten(0, 1).contiguous()

In [9]:
seed = 617 # на своё усмотрение можно выбрать другой seed
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.benchmark = True

model = NatureModel(*obs.shape, n_actions, device)
model.to(device)
policy = Policy(model)
runner = EnvRunner(env, policy, nsteps=5, # nsteps -- длина частичной траектории
                                          # уменьшение nsteps может привести к вынужденному увеличению
                                          # количества итераций оптимизации
                   transforms=[ComputeValueTargets(policy=policy, device=device),
                               MergeTimeBatch()])

Настало время реализовать сам алгоритм Advantage-Actor Critic (A2C). Его псевдокод можно посмотреть в [конспектах лекций (Раздел 5.2.5)](https://arxiv.org/pdf/2201.09746.pdf), в публикции [Mnih et al. 2016](https://arxiv.org/abs/1602.01783) и в [лекции](https://www.youtube.com/watch?v=Tol_jw5hWnI&list=PLkFD6_40KJIxJMR-j5A1mkxK26gh_qg37&index=20) Сергея Левина.

In [10]:
def explained_variance(y_pred: np.ndarray, y_true: np.ndarray) -> np.ndarray:
    """
    Computes fraction of variance that ypred explains about y.
    Returns 1 - Var[y-ypred] / Var[y]

    interpretation:
        ev=0  =>  might as well have predicted zero
        ev=1  =>  perfect prediction
        ev<0  =>  worse than just predicting zero

    :param y_pred: the prediction
    :param y_true: the expected value
    :return: explained variance of ypred and y
    """
    assert y_true.ndim == 1 and y_pred.ndim == 1
    var_y = np.var(y_true)
    return np.nan if var_y == 0 else 1 - np.var(y_true - y_pred) / var_y

class A2C:
    def __init__(
        self,
        policy,
        optimizer,
        value_loss_coef=0.25,
        entropy_coef=1e-2,
        max_grad_norm=0.5,
    ):
        self.policy = policy
        self.optimizer = optimizer
        self.value_loss_coef = value_loss_coef
        self.entropy_coef = entropy_coef
        self.max_grad_norm = max_grad_norm

    def policy_loss(self, trajectory):
        # Тут нужно вычислить Advantages.
        advantages = trajectory["value_targets"] - trajectory["values"].detach().clone()
        trajectory["advantages"] = advantages
        trajectory["policy_loss"] = -torch.mean(trajectory["log_probs"] * advantages)

    def value_loss(self, trajectory):
        trajectory["value_loss"] = F.mse_loss(
            trajectory["values"], trajectory["value_targets"]
        )

    def loss(self, trajectory):
        self.policy_loss(trajectory)
        self.value_loss(trajectory)
        trajectory["entropy_loss"] = -torch.mean(trajectory["entropy"])
        trajectory["loss"] = (
            trajectory["policy_loss"]
            + self.entropy_coef * trajectory["entropy_loss"]
            + self.value_loss_coef * trajectory["value_loss"]
        )

    def step(self, trajectory):
        self.loss(trajectory)
        self.optimizer.zero_grad()
        trajectory["loss"].backward()
        total_norm = 0.0
        for p in self.policy.model.parameters():
            if not p.requires_grad:
                continue
            param_norm = p.grad.detach().data.norm(2)
            total_norm += param_norm.item() ** 2
        trajectory["grad_norm"] = total_norm ** 0.5
        trajectory["explained_variance"] = explained_variance(trajectory["values"].detach().cpu().numpy(), trajectory["value_targets"].detach().cpu().numpy())
        torch.nn.utils.clip_grad_norm_(self.policy.model.parameters(), self.max_grad_norm)
        self.optimizer.step()

In [11]:
10_000_000 / 40

250000.0

Теперь можно непосредственно обучить Вашу модель. С разумно подобранными гиперпараметрами обучение на одной GTX1080 на протяжении 10 миллионов шагов суммарно со всех батчированных сред (что переводится примерно в 5 часов работы) должно быть возможно достигнуть *среднюю исходную награду за 100 последних эпизодов* (значение переменной в `Tensorboard` по ключу `reward_mean_100`, усреднение берётся по 100 последним эпизодам в каждой среде в батче) **не меньше 600**. Это и будет считаться успешным результатом работы алгоритма `A2C`.

**Внимание!** При *коректной* имплементации алгоритма *обоснованное* преодоление порога **400** по `reward_mean_100` *транслируется в* **8 баллов за задание**, *обоснованное* преодоление порога **600** по `reward_mean_100` на *корректно* написанном алгоритме обучения `A2C` *транслируется в* **10 баллов за задание**.

Вам так же, по возможности, рекомендуется отобразить данную величину относительно `runner.step_var` &mdash; количества взаимодействий со всеми средами. Также очень рекомендуется предоставить графики следующих показателей (полезно для отладки кода):
* [Коэффициент детерминации](https://en.wikipedia.org/wiki/Coefficient_of_determination) между целевыми значениями ценности и их предсказаниями
* Энтропия политики $\pi$
* Функция потерь ценности (Value loss)
* Функция потерь политики (Policy loss)
* Целевые значения ценности (Value targets)
* Предсказания значений ценности (Value predictions)
* Норма градиента
* Advantages
* Общая функция потерь (A2C loss)

В качестве оптимизатора рекомендуется взять метод [RMSProp](https://pytorch.org/docs/stable/generated/torch.optim.RMSprop.html) с [линейным убыванием шага](https://pytorch.org/docs/stable/generated/torch.optim.lr_scheduler.LambdaLR.html), начиная с 7e-4 до 0, константой сглаживания (alpha в PyTorch и decay в TensorFlow), равной 0.99 и epsilon, равным 1e-5.

Запуск Tensorboard

In [12]:
def _get_linear_schedule_with_warmup_lr_lambda(
    current_step: int, *, num_warmup_steps: int, num_training_steps: int
):
    if current_step < num_warmup_steps:
        return float(current_step) / float(max(1, num_warmup_steps))
    return max(
        0.0,
        float(num_training_steps - current_step)
        / float(max(1, num_training_steps - num_warmup_steps)),
    )


def get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps, num_training_steps, last_epoch=-1
):
    """
    Create a schedule with a learning rate that decreases linearly from the initial lr set in the optimizer to 0, after
    a warmup period during which it increases linearly from 0 to the initial lr set in the optimizer.

    Args:
        optimizer ([`~torch.optim.Optimizer`]):
            The optimizer for which to schedule the learning rate.
        num_warmup_steps (`int`):
            The number of steps for the warmup phase.
        num_training_steps (`int`):
            The total number of training steps.
        last_epoch (`int`, *optional*, defaults to -1):
            The index of the last epoch when resuming training.

    Return:
        `torch.optim.lr_scheduler.LambdaLR` with the appropriate schedule.
    """

    lr_lambda = partial(
        _get_linear_schedule_with_warmup_lr_lambda,
        num_warmup_steps=num_warmup_steps,
        num_training_steps=num_training_steps,
    )
    return LambdaLR(optimizer, lr_lambda, last_epoch)

In [13]:
N_TRAIN_STEPS = 250_000
optimizer = opt.RMSprop(model.parameters(), lr=7e-4, alpha=0.99, eps=1e-5)
scheduler = get_linear_schedule_with_warmup(optimizer, 0, N_TRAIN_STEPS)
a2c = A2C(policy=policy, optimizer=optimizer)

for i in tqdm(range(N_TRAIN_STEPS)):
    trajectory = runner.get_next()
    a2c.step(trajectory)
    runner.add_summary("Losses/loss", trajectory["loss"].item())
    runner.add_summary("Losses/policy_loss", trajectory["policy_loss"].item())
    runner.add_summary("Losses/value_loss", trajectory["value_loss"].item())
    runner.add_summary("Values/predicted", torch.mean(trajectory["values"].detach().cpu()).item())
    runner.add_summary("Values/targets", torch.mean(trajectory["value_targets"].detach().cpu()).item())
    runner.add_summary("Values/entropy", torch.mean(trajectory["entropy"].detach().cpu()).item())
    runner.add_summary("Values/advantages", torch.mean(trajectory["advantages"].detach().cpu()).item())
    runner.add_summary("grad_norm", trajectory["grad_norm"])
    runner.add_summary("Values/explained_variance", trajectory["explained_variance"])
    runner.add_summary("lr", scheduler.get_last_lr()[0])
    scheduler.step()
    if (i + 1) % 25000 == 0:
        with open(f"./checkpoints/step_{i}.pt", "wb+") as f:
            torch.save(model.state_dict(), f)
with open(f"./checkpoints/final.pt", "wb+") as f:
    torch.save(model.state_dict(), f)

  0%|          | 0/250000 [00:00<?, ?it/s]

In [14]:
test_env = nature_dqn_env("SpaceInvadersNoFrameskip-v4", nenvs=None, clip_reward=False, summaries=False)

A.L.E: Arcade Learning Environment (version 0.8.1+53f58b7)
[Powered by Stella]


In [15]:
n_lives = 3

def evaluate(env, agent, n_games=1, greedy=False, t_max=10000):
    """
    Играем n_games игр до конца.
    В случае жадной политики, выбираем действия как argmax(qvalues).
    Возвращаем среднюю награду.
    """
    rewards = []
    for _ in range(n_games):
        s, _ = env.reset()
        reward = 0
        for _ in range(t_max):
            output = agent.act([s])
            action = output['logits'].argmax(dim=-1).item() if greedy else output['actions'][0]
            s, r, terminated, truncated, _ = env.step(action)
            reward += r
            if terminated or truncated:
                break

        rewards.append(reward)
    return np.mean(rewards)

In [16]:
# запись эпизодов
with RecordVideo(
    env=test_env,
    video_folder="./videos/False",
    episode_trigger=lambda episode_number: True
) as env_monitor:
    sessions = [evaluate(env_monitor, policy, n_games=n_lives, greedy=False) for _ in range(1)]

/home/vscode/.local/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:335: UserWarning: WARN: No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps.
  logger.warn(


Moviepy - Building video /home/imumtozee/workspace/MIPT_RL/task4/videos/False/rl-video-episode-0.mp4.
Moviepy - Writing video /home/imumtozee/workspace/MIPT_RL/task4/videos/False/rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /home/imumtozee/workspace/MIPT_RL/task4/videos/False/rl-video-episode-0.mp4
Moviepy - Building video /home/imumtozee/workspace/MIPT_RL/task4/videos/False/rl-video-episode-1.mp4.
Moviepy - Writing video /home/imumtozee/workspace/MIPT_RL/task4/videos/False/rl-video-episode-1.mp4



Moviepy - Done !
Moviepy - video ready /home/imumtozee/workspace/MIPT_RL/task4/videos/False/rl-video-episode-1.mp4
Moviepy - Building video /home/imumtozee/workspace/MIPT_RL/task4/videos/False/rl-video-episode-2.mp4.
Moviepy - Writing video /home/imumtozee/workspace/MIPT_RL/task4/videos/False/rl-video-episode-2.mp4



Moviepy - Done !
Moviepy - video ready /home/imumtozee/workspace/MIPT_RL/task4/videos/False/rl-video-episode-2.mp4


In [17]:
video_paths = sorted([s for s in Path('videos', 'False').iterdir() if s.suffix == '.mp4'])
video_path = video_paths[1]
data_url = str(video_path)

HTML("""
<video width="640" height="480" controls>
  <source src="{}" type="video/mp4">
</video>
""".format(data_url))

In [18]:
# запись эпизодов
with RecordVideo(
    env=test_env,
    video_folder="./videos/True",
    episode_trigger=lambda episode_number: True
) as env_monitor:
    sessions = [evaluate(env_monitor, policy, n_games=n_lives, greedy=True) for _ in range(1)]

Moviepy - Building video /home/imumtozee/workspace/MIPT_RL/task4/videos/True/rl-video-episode-0.mp4.
Moviepy - Writing video /home/imumtozee/workspace/MIPT_RL/task4/videos/True/rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /home/imumtozee/workspace/MIPT_RL/task4/videos/True/rl-video-episode-0.mp4
Moviepy - Building video /home/imumtozee/workspace/MIPT_RL/task4/videos/True/rl-video-episode-1.mp4.
Moviepy - Writing video /home/imumtozee/workspace/MIPT_RL/task4/videos/True/rl-video-episode-1.mp4



Moviepy - Done !
Moviepy - video ready /home/imumtozee/workspace/MIPT_RL/task4/videos/True/rl-video-episode-1.mp4
Moviepy - Building video /home/imumtozee/workspace/MIPT_RL/task4/videos/True/rl-video-episode-2.mp4.
Moviepy - Writing video /home/imumtozee/workspace/MIPT_RL/task4/videos/True/rl-video-episode-2.mp4



Moviepy - Done !
Moviepy - video ready /home/imumtozee/workspace/MIPT_RL/task4/videos/True/rl-video-episode-2.mp4


In [19]:
video_paths = sorted([s for s in Path('videos', 'True').iterdir() if s.suffix == '.mp4'])
video_path = video_paths[1]
data_url = str(video_path)

HTML("""
<video width="640" height="480" controls>
  <source src="{}" type="video/mp4">
</video>
""".format(data_url))

# Логи обучения

## 1. Средняя длина эпизодов

<img src="./img/1.png" width=640/>

В данном графике можно увидеть, что по ходу обучения, эпизоды в среднем становятся длинее.

## 2. Максимальный реворд со всех сред

<img src="./img/2.png" width=640/>

Можно увидеть, что по ходу обучения максимальный реворд среди всех параллельных сред растет.

## 3. Минимальный реворд со всех сред

<img src="./img/3.png" width=640/>

Можно увидеть, что по ходу обучения минимальный реворд среди всех параллельных сред тоже растет.

## 4. Средний реворд за последние 100 эпизодов

<img src="./img/4.png" width=640/>

Основная метрика успеха в задании. Рост минимальных и максимальных ревордов со всех параллельных сред спустя некоторое время отражатеся на данном показателе и он положительно коррелирует с двумя графиками выше.

## 5. Средний реворд за последний эпизод

<img src="./img/5.png" width=640/>

Значение реворда за последний эпизод усредненное по всем параллельным средам. Можно увидеть что все графики относительно эпизода положительно коррелируют между собой.

## 6. A2C Loss

<img src="./img/6.png" width=640/>

На этом графике можно увидеть более менее стабильное течение обучения, то есть нет сильных осцилляций и модель следует exploration-exploitation tradeoff.

## 7. Actor Loss

<img src="./img/7.png" width=640/>

График показывает логарифм правдоподобия действий умноженных на advantage который они дают.

## 8. Critic Locc

<img src="./img/8.png" width=640/>

График показывает среднюю квадратическую ошибку между value функцией предсказанной моделью и правильной (которая тоже все равно использует предсказания модели)

## 9. Advantages

<img src="./img/9.png" width=640/>

Разница между правильными значениями ценности состояний и теми которые предсказала модель.

## 10. Entropy Loss

<img src="./img/10.png" width=640/>

Средняя энтропия действий. Добавляется к общему лоссу как регуляризатор

## 11. Коэффициент детерминации

<img src="./img/11.png" width=640/>

Доля дисперсии в распределении правильных значений ценности состояний объясняемой дисперсией предсказанных ценностей.

## 12. Предсказанные значения функции ценности

<img src="./img/12.png" width=640/>

Наблюдается рост значений по ходу обучения.

## 13. Целевые значения функции ценности

<img src="./img/13.png" width=640/>

Так как при вычислении целевых значений функции ценности состояний используются предсказания модели, графики сильно похожи друг на друга.

## 14. Норма градиентов

<img src="./img/14.png" width=640/>

Норма градиентов весов до операции clip. Можно увидеть что по ходу обучения норма в среднем составляет то значение по которому и прозводится clip: 0.5

# Информация об обучении

Прикрепление скриншотов графиков обучения модели в `Tensorboard` ниже является обязательным. Для доступа к `Tensorboard` запустите из командной строки в одной директории с данным ноутбуком следующую команду:
```
tensorboard --logdir logs --port 6006
```
В результате вывод в командную строку укажет, по какому адресу можно подсоединиться к инстанции `Tensorboard`, например, по адресу `http://localhost:6006/`. Оттуда можно и сделать скриншоты, демонстрирующие результаты обучения модели. Сами скриншоты с именем файла `image_name_x.png` для удобства лучше сохранить в директорию `./img`, откуда можно легко их прикреплять в `Markdown-клетках` ниже по команде со следующей конструкцией:
```
<img src=./img/image_name_x.png width=640>
```
Тут также требуется подписать изображения и дать небольшой комментарий по каждому скриншоту, что на нём описано.

**Внимание!** В случае перезапуска процедуры обучения модели рекомендуется удалить директорию `./logs` вместе с её содержимым перед непосредственным перезапуском, чтобы не испортить отображающиеся графики в `Tensorboard`.

**Совет.** При работе в Google Colab можно просто скачать директорию `./logs` и уже локально запустить `Tensorboard` для снятия скриншотов. Также можно обученного агента сохранить, скачать и локально на cpu запустить для записи роликов (для этого понадобится самостоятельно прописать код сохранения и загрузки модели в ноутбук из [файла](https://pytorch.org/tutorials/beginner/saving_loading_models.html)).

**Внимание!** Посылку для сдачи задания требуется оформить в виде `.zip` архива, в котором будут *данный ноутбук*, использованные для его работы *скрипты*, *директории* `./videos`, `./logs` и `./img` с содержимым. Только так и не иначе!

__Вставьте в данную ячейку свой ответ, подкреплённый скриншотами__